In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, LlamaTokenizer
import random
from tqdm import tqdm
import re, torch
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from models.modelings_alignable_llama import *
from utils.train_utils import *
CACHE_DIR = "../../.cache/"

#### GPT-J Alpaca 6B

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "nlpcloud/instruct-gpt-j-fp16",
    cache_dir=CACHE_DIR,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(
    "nlpcloud/instruct-gpt-j-fp16",
    cache_dir=CACHE_DIR
)

In [ ]:
prompt = """Reason True or False for the following statement.\n3 is equal to 1.\n"""
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to("cuda")
model.eval()
outputs = model(
    input_ids,
    attention_mask=attention_mask
)
pred_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
generated_tokens = tokenizer.decode(pred_labels[0])
generated_tokens

#### StableLM 7B

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(
#     "../../stablelm_7b/",
#     cache_dir=CACHE_DIR,
#     torch_dtype=torch.bfloat16
# )
# _ = model.to("cuda")
# tokenizer = AutoTokenizer.from_pretrained(
#     "../../stablelm_7b/",
#     cache_dir=CACHE_DIR
# )

#### Loading Dolly-12B

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(
#     "databricks/dolly-v2-12b",
#     cache_dir=CACHE_DIR,
#     torch_dtype=torch.bfloat16
# )
# _ = model.to("cuda")
# tokenizer = AutoTokenizer.from_pretrained(
#     "databricks/dolly-v2-12b",
#     cache_dir=CACHE_DIR
# )

#### Loading Alpaca-13B

In [ ]:
alignment_config = {
    'layer': 25,
    "token_range" : [0, 1]
}
model = AlignableLlamaForCausalLM.from_pretrained(
    "../../alpaca_13b/",
    alignment_config=alignment_config,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = LlamaTokenizer.from_pretrained(
    "chavinlo/alpaca-13b",
    cache_dir=CACHE_DIR
)

#### Loading Alpaca-7B

In [ ]:
alignment_config = {
    'layer': 25,
    "token_range" : [0, 1]
}
model = AlignableLlamaForCausalLM.from_pretrained(
    "../../alpaca_7b_bf16/",
    alignment_config=alignment_config,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="../../alpaca_7b/",
    cache_dir=CACHE_DIR
)

In [ ]:
alpaca_prompt_template = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Input:
3 1 2

### Response:
No, since the largest number is 3, and the smallest number is 1, so the difference is 3 - 1 = 2, 2 is smaller than 3.

### Input:
7 2 4

### Response:
Yes, since the largest number is 7, and the smallest number is 2, so the difference is 7 - 2 = 5, 5 is larger than 3.

### Input:
%s

### Response:
"""

alpaca_prompt_template_no_inputs = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Response:
"""

alpaca_instruction = """Difference between the largest number and the smallest number is bigger than 3."""
prompt = alpaca_prompt_template % (alpaca_instruction, "2 9 4")
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to("cuda")
model.eval()
outputs = model(
    input_ids,
    attention_mask=attention_mask
)
pred_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
generated_tokens = tokenizer.decode(pred_labels[0])
generated_tokens

In [ ]:
prompt

#### Working task: Pricing Tag Game

In [ ]:
def factual_sampler(
    lower_bound, upper_bound,
    tokenizer,
    min_lower_bound=0.00, max_upper_bound=10.00,
):
    instruction = f"Please say yes only if it costs between {lower_bound} and {upper_bound} dollars, otherwise no."
    
    all_input_ids = []
    all_output_ids = [] # this one does not have input ids, etc..
    all_attention_mask = []
    
    result = []
    current = min_lower_bound
    step = 0.01
    raw_amounts = []
    while current <= max_upper_bound:
        amount = round(current, 2)
        if amount >= lower_bound and amount <= upper_bound:
            label = tokenizer.convert_tokens_to_ids("Yes")
        else:
            label = tokenizer.convert_tokens_to_ids("No")
            
        amount = "%.2f dollars" % amount
        prompt = alpaca_prompt_template % (instruction, amount)

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids[0]
        attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask[0]
        output_ids = (torch.ones(input_ids.shape[0])*-100).long().tolist()
        output_ids[-1] = label
        input_ids = input_ids.tolist()
        
        all_input_ids += [input_ids]
        all_attention_mask += [attention_mask]
        all_output_ids += [output_ids]
        raw_amounts += [current]
        
        current += step

    return all_input_ids, all_attention_mask, all_output_ids, raw_amounts

In [ ]:
raw_prealign = factual_sampler(
    3.50, 8.50,
    tokenizer
)
prealign_dataset = Dataset.from_dict(
    {
        "input_ids": raw_prealign[0], 
        "attention_mask": raw_prealign[1], 
        "labels": raw_prealign[2],
    }
).with_format("torch")
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [ ]:
total_count = 0
correct_count = 0
model.eval()
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        outputs = model(
            input_ids=inputs['input_ids'],
            labels=inputs['labels'],
            attention_mask=inputs['attention_mask']
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

#### Working task: Pricing Tag Game with Material Type

In [ ]:
examples = []
for i in range(100, 1000):
    examples += [{
        'a' : f"gold {str(i)[0]}.{str(i)[1]}{str(i)[2]} dollars",
        'output' : ['Yes', 'yes'] if i < 700 and i > 200 else ['No', 'no']
        # 'output' : ['No', 'no']
    }]

In [ ]:
# instruction = """Please say yes only if wood costs between 3.00 and 4.99 dollars, otherwise no."""
# instruction = """Please say yes only if metal costs between 2.00 and 6.99 dollars, otherwise no."""
alpaca_prompt_template = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Input:
%s

### Response:
"""

In [ ]:
n_correct = 0
model.eval()
pred_labels = []
with torch.no_grad():
    for example in tqdm(examples):
        inputs = example['a']
        alpaca_prompt = alpaca_prompt_template % (instruction, inputs)

        input_ids = tokenizer(alpaca_prompt, return_tensors="pt").input_ids.to("cuda")
        generated_ids = model.generate(
            input_ids, do_sample=True, 
            temperature=0.01, max_length=input_ids[0].shape[0]+3,
        )
        generated_tokens = tokenizer.decode(generated_ids[0])
        pred_label = generated_tokens.split("</s>")[0].strip(".\n").split("\n")[-1].strip()
        correct_label = example['output']
        if pred_label in correct_label:
            n_correct += 1
        print(pred_label, correct_label, inputs)
        pred_labels += [pred_label]
        

In [ ]:
n_correct/len(examples)

In [ ]:
plot_data_x = []
plot_data_y = []
for i in range(len(pred_labels)):
    plot_data_x += [round((i+100)/100.0, 2)]
    plot_data_y += [0 if pred_labels[i] in ['No', 'no'] else 1]

In [ ]:
plt.figure(figsize=(10,1))
plt.plot(plot_data_x, plot_data_y, '-')
plt.show()